In [2]:
import utils, sift
from jit_ransac import ransac, ratio_test_threshold_match
from torchvision.io import read_image, ImageReadMode
import torchvision.transforms.v2 as T
from PIL import Image
import torch
from tqdm.auto import tqdm

gen = torch.manual_seed(19973532)

c:\Users\sambi\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: 'Could not find module 'C:\Users\sambi\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
c:\Users\sambi\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\datapoints\__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pyt

'NoneType' object has no attribute 'cadam32bit_grad_fp32'


In [15]:
resize = T.Resize(256, antialias=True)
transforms = T.Compose(
    [
        T.Resize(256, antialias=True),
        T.RandomRotation(30, interpolation=Image.BILINEAR),
        T.RandomAffine(0, translate=(0.2, 0.2), interpolation=Image.BILINEAR),
        T.RandomPerspective(distortion_scale=0.5),
        T.ToTensor(),
    ]
)

c:\Users\sambi\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\transforms\v2\_deprecated.py:41: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `transforms.Compose([transforms.ToImageTensor(), transforms.ConvertImageDtype()])`.
  warnings.warn(


In [16]:
from tensordict import tensorclass, MemmapTensor
@tensorclass
class KeyPoints:
    pts: torch.Tensor
    desc: torch.Tensor

    @classmethod
    def from_images(cls, images: torch.Tensor, num_features=512, patch_size=41, angle_bins=8, spatial_bins=8, batch_size: int = 8, device: str = 'cuda'):
        descs = MemmapTensor(images.size(0), num_features, 8 * angle_bins * spatial_bins, dtype=torch.float32)
        kpts = MemmapTensor(images.size(0), num_features, 2, dtype=torch.float32)

        model = sift.SIFT(num_features=num_features, patch_size=patch_size, angle_bins=angle_bins, spatial_bins=spatial_bins)
        model.to(device)
        with torch.no_grad():
            for i in tqdm(range(0, images.size(0), batch_size)):
                batch = images[i:i+batch.size].to(device)
                k, d = model(batch)
                descs[i:i+batch_size] = d.to('cpu')
                kpts[i:i+batch_size] = k.to('cpu')
        return cls(kpts, descs, batch_size=[images.size(0)])

In [21]:
example_image_path = "example.jpg"
image = Image.open(example_image_path).convert("RGB")

index_image = resize(image)
query_image = transforms(image)
index_image_tensor = torch.unsqueeze(T.ToTensor()(index_image), 0)
query_image_tensor = torch.unsqueeze(T.ToTensor()(query_image), 0)

# Extract keypoints
index = KeyPoints.from_images(index_image_tensor, batch_size=1)
query = KeyPoints.from_images(query_image_tensor, batch_size=1)

# Visualize keypoints
index_kps_image = utils.visualize_keypoints(index_image, index.pts)
query_kps_image = utils.visualize_keypoints(query_image, query.pts)

# Concatenate and display the images side by side
side_by_side = utils.concat(index_kps_image, query_kps_image, dim=1)
side_by_side.show()

AssertionError: Torch not compiled with CUDA enabled